In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
with open("/content/drive/My Drive/Colab Notebooks/.../data/bps.csv", "r") as f:
  df = pd.read_csv(f, header=0, index_col='MONTH', dtype='int64', parse_dates=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, January to December
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2008    12 non-null     int64
 1   2009    12 non-null     int64
 2   2010    12 non-null     int64
 3   2011    12 non-null     int64
 4   2012    12 non-null     int64
 5   2013    12 non-null     int64
 6   2014    12 non-null     int64
 7   2015    12 non-null     int64
 8   2016    12 non-null     int64
 9   2017    12 non-null     int64
 10  2018    12 non-null     int64
 11  2019    12 non-null     int64
 12  2020    12 non-null     int64
 13  2021    12 non-null     int64
dtypes: int64(14)
memory usage: 1.4+ KB


In [ ]:
df


,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
MONTH,,,,,,,,,,,,,,
January,437966,473165,493799,548821,652692,614328,753079,724698,814303,1107968,1097839,1201735,1290411,137230
February,465449,421555,523135,568057,592502,678415,702666,794302,888309,1023388,1197503,1243996,872765,115765
March,502041,511314,594242,598068,658602,725316,765607,792804,915019,1059777,1363426,1311911,486155,130933
April,459129,487121,555915,608093,626100,646117,726332,750999,901095,1171386,1302321,1274231,158066,125001
May,508955,521735,600031,600191,650883,700708,752363,794294,915206,1148588,1242705,1249536,161842,152604
Juny,529064,550582,613422,674402,695531,789594,851475,815307,857651,1144001,1322674,1434103,156561,137247
July,567364,593415,658476,745451,701200,717784,777210,815351,1032741,1370591,1547231,1468173,155742,135438
Augus,599506,566797,586530,621084,634194,771009,826821,853244,1031986,1393243,1511021,1530268,161549,124751
September,501018,493799,560367,650071,683584,770878,791296,870351,1006653,1250231,1370943,1388719,148984,124071


In [ ]:
import pandas as pd
import os
with open("/content/drive/My Drive/Colab Notebooks/.../data/bps-fix.csv", "r") as f:
  series = pd.read_csv(f, index_col='MONTH', header=0)
series.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, 2008-01 to 2021-12
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   total   168 non-null    int64
dtypes: int64(1)
memory usage: 2.6+ KB


In [ ]:
series

,total
MONTH,
2008-01,437966
2008-02,465449
2008-03,502041
2008-04,459129
2008-05,508955
...,...
2021-08,124751
2021-09,124071
2021-10,148645


In [ ]:
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

In [ ]:
# create a differenced series/bagi dataset
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

In [ ]:
# invert differenced value/evaluasi
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

In [ ]:
# scale train and test data to [-1, 1]/preprocessing
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

In [ ]:
# inverse scaling for a forecasted value/denormalisasi
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

In [ ]:
# fit an LSTM network to training data/LSTM
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, activation='tanh', batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    model.fit(X, y, epochs=nb_epoch, batch_size=batch_size, verbose=1)
    return model

In [ ]:
# make a one-step forecast/pengujian
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [ ]:
# transform data to be stationary
from pandas import Series, DataFrame
raw_values = series.total.values.astype('int64')
diff_values = difference(raw_values, 1)
print('Dataset \n',diff_values)

Dataset 
 0      27483
1      36592
2     -42912
3      49826
4      20109
       ...  
162   -10687
163     -680
164    24574
165     4554
166    10420
Length: 167, dtype: int64


In [ ]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values
print('Supervised Learning Input, Output \n',supervised_values)
print('Dataset :', len(supervised_values))

NameError: ignored